In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import time

In [2]:
net = "Varus"
url = "https://varus.ua"
curr_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
time_sleep = 20
# Empty data list to store product cards
data = []

In [3]:
driver = webdriver.Firefox()
driver.get(url)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "swl__address")))
print("Page loaded successfully")

Page loaded successfully


In [4]:
# Блок адресів
address_button = driver.find_element(By.CSS_SELECTOR, "p.swl__address")
address_button.click()

In [5]:
# Блок самовивозу
address_block = driver.find_element(By.CSS_SELECTOR, "div.radio-switcher-square")
self_delivery = address_block.find_elements(By.CSS_SELECTOR, "div.radio-switcher-square__input")[1]
self_delivery.click()

In [6]:
# Вибір міста
city_block = driver.find_element(By.CSS_SELECTOR, "div.m-input-autocomplete__chevron")
city_block.click()

In [7]:
# Знаходження переліку міст та клік по місту Київ
city_list = driver.find_elements(By.CSS_SELECTOR, "li.m-input-autocomplete__autocomplete-item")
for city in city_list:
    if city.text == "Київ":
        city.click()
        break

In [8]:
# Вибір магазину
shop_list = driver.find_element(By.CSS_SELECTOR, "div.m-locate-switcher-shop__smooth")
shop_list.click()

In [9]:
# Знаходження переліку магазинів та клік по "вул. Драгоманова 40г"
shop_list =shop_list.find_elements(By.CSS_SELECTOR, "li.m-input-autocomplete__autocomplete-item")
for shop in shop_list:
    if shop.text == "вул. Драгоманова 40г":
        shop.click()
        break

In [10]:
# До покупок
button_to_shop = driver.find_element(By.CSS_SELECTOR, "div.shp-save")
if button_to_shop.text == "До покупок":
    button_to_shop.click()  

In [11]:
# Прохід по всім категоріям продукції та збір цін
category_url = "https://varus.ua/torti"
driver.get(category_url)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "swl__address")))
print("Page loaded successfully")

Page loaded successfully


In [12]:
# Скролінг сторінки для розкриття всіх товарів
while True:
    try:
        print("Start scrolling")
        driver.find_element(By.CSS_SELECTOR, "div.bottom-plp-block").find_element(By.TAG_NAME, "button").click()
        print("Scroll completed")
        time.sleep(time_sleep)
        print("Sleep finished")
    except:
        break

Start scrolling
Scroll completed
Sleep finished
Start scrolling
Scroll completed
Sleep finished
Start scrolling
Scroll completed
Sleep finished
Start scrolling


In [13]:
# Get all elements from the current page
elems = driver.find_element(By.CLASS_NAME, "products")

# Find all product cards
product_cards = elems.find_elements(By.CLASS_NAME, "sf-product-card")
product_cards_list = [prod_card for prod_card in product_cards if prod_card.is_displayed()]

# Loop by product_cards_list
for product in product_cards_list:
    product_name = product.find_element(By.CLASS_NAME, "sf-product-card__title").text
    product_quantity = product.find_element(By.CLASS_NAME, "sf-product-card__quantity").text
    try:
        product_regular_price = product.find_element(By.CLASS_NAME, "sf-price__regular").text.split(" ")[0]
    except:
        product_regular_price = ''
    try:
        product_old_price = product.find_element(By.CLASS_NAME, "sf-price__old").text
    except:
        product_old_price = ''
    try:
        product_special_price = product.find_element(By.CLASS_NAME, "sf-price__special").text.split(" ")[0]
    except:
        product_special_price = ''
    try:
        product_special_text = product.find_element(By.CLASS_NAME, "sf-product-card__badge_text").text
    except:
        product_special_text = ''
    data.append({
        'net': net,
        'name': product_name,
        'quantity': product_quantity,
        'regular_price': product_regular_price,
        'old_price': product_old_price,
        'special_price': product_special_price,
        'special_text': product_special_text,
        'curr_time': curr_time
    })

In [14]:
driver.close()

In [15]:
# Specify the CSV file name
filename = "data.csv"

# Extract fieldnames from the first dictionary
fieldnames = data[0].keys()

# Write to CSV
with open(filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # write header row
    writer.writerows(data)  # write data rows